In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import (VGG16, VGG19, ResNet50, InceptionV3, MobileNetV2,
                                DenseNet121, Xception)
from keras.applications.efficientnet import (EfficientNetB0, EfficientNetB1, EfficientNetB2,
                                             EfficientNetB3, EfficientNetB4, EfficientNetB5,
                                             EfficientNetB6, EfficientNetB7, preprocess_input as efficientnet_preprocess_input)
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input
from keras.applications.inception_v3 import preprocess_input as inceptionv3_preprocess_input
from keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess_input
from keras.applications.densenet import preprocess_input as densenet_preprocess_input
from keras.applications.xception import preprocess_input as xception_preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.layers import BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
import seaborn as sns
from keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Define your data directories
train_folder = '/kaggle/input/chest-ctscan-images/Data/train'
valid_folder = '/kaggle/input/chest-ctscan-images/Data/valid'
test_folder = '/kaggle/input/chest-ctscan-images/Data/test'

In [4]:
# Model configurations
num_classes = 4
batch_size = 32
epochs = 100

In [5]:
# Map model names to their function, preprocess_input, and input_shape
model_info = {
    'VGG16': (VGG16, vgg16_preprocess_input, (224, 224, 3)),
    'VGG19': (VGG19, vgg19_preprocess_input, (224, 224, 3)),
    'ResNet50': (ResNet50, resnet50_preprocess_input, (224, 224, 3)),
    'InceptionV3': (InceptionV3, inceptionv3_preprocess_input, (299, 299, 3)),
    'MobileNetV2': (MobileNetV2, mobilenetv2_preprocess_input, (224, 224, 3)),
    'DenseNet121': (DenseNet121, densenet_preprocess_input, (224, 224, 3)),
    'Xception': (Xception, xception_preprocess_input, (299, 299, 3)),
    'EfficientNetB0': (EfficientNetB0, efficientnet_preprocess_input, (224, 224, 3)),
    'EfficientNetB1': (EfficientNetB1, efficientnet_preprocess_input, (240, 240, 3)),
    'EfficientNetB2': (EfficientNetB2, efficientnet_preprocess_input, (260, 260, 3)),
    'EfficientNetB3': (EfficientNetB3, efficientnet_preprocess_input, (300, 300, 3)),
    'EfficientNetB4': (EfficientNetB4, efficientnet_preprocess_input, (380, 380, 3)),
    'EfficientNetB5': (EfficientNetB5, efficientnet_preprocess_input, (456, 456, 3)),
    'EfficientNetB6': (EfficientNetB6, efficientnet_preprocess_input, (528, 528, 3)),
    'EfficientNetB7': (EfficientNetB7, efficientnet_preprocess_input, (600, 600, 3))
}


In [6]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [7]:
def train_model(model_name, num_classes, train_folder, valid_folder, epochs, batch_size):
    model_class, preprocess_input, input_shape = model_info[model_name]

    # Data generators with model-specific preprocessing
    train_datagen = ImageDataGenerator(
        dtype='float32',
        preprocessing_function=preprocess_input,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False
    )
    val_datagen = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(
        dtype='float32',
        preprocessing_function=preprocess_input,
    )
    train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical'
    )
    validation_generator = val_datagen.flow_from_directory(
        valid_folder,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    # Model creation
    base_model = model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define Callbacks
    checkpoint = ModelCheckpoint(filepath=f'{model_name}_model.h5',
                                 monitor='val_accuracy',
                                 mode='max',
                                 save_best_only=True,
                                 verbose=1)

    earlystop = EarlyStopping(monitor='val_accuracy',
                              min_delta=0.001,
                              patience=15,
                              restore_best_weights=True)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=10,
                                  verbose=1,
                                  min_delta=0.0001,
                                  min_lr=0.0001)

    callbacks = [checkpoint, earlystop, reduce_lr]

    # Train the model
    history = model.fit(train_generator, validation_data=validation_generator, callbacks=callbacks, epochs=epochs, steps_per_epoch=np.ceil(train_generator.samples/batch_size), validation_steps=np.ceil(validation_generator.samples/batch_size))
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Save the plot
    plt.savefig(f'{model_name}_accuracy_loss_plot.png')
    plt.close()
    
    return model


In [8]:
# Train and save each model
for model_name in model_info.keys():
    print(f"Training {model_name}...")
    model = train_model(model_name, num_classes, train_folder, valid_folder, epochs, batch_size)
#     model.save(f'{model_name}_model.h5')
    print(f"{model_name} model training completed and saved.")
    

Training VGG16...
Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/100


2023-12-25 12:59:00.383732: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


20/20 [==============================] - ETA: 0s - loss: 1.3725 - accuracy: 0.4144
Epoch 1: val_accuracy improved from -inf to 0.11111, saving model to VGG16_model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 44s 931ms/step - loss: 1.3725 - accuracy: 0.4144 - val_loss: 129.6460 - val_accuracy: 0.1111 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 1.3255 - accuracy: 0.4568
Epoch 2: val_accuracy improved from 0.11111 to 0.18056, saving model to VGG16_model.h5
20/20 [==============================] - 12s 560ms/step - loss: 1.3255 - accuracy: 0.4568 - val_loss: 22.9139 - val_accuracy: 0.1806 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 1.2887 - accuracy: 0.4617
Epoch 3: val_accuracy improved from 0.18056 to 0.48611, saving model to VGG16_model.h5
20/20 [==============================] - 12s 593ms/step - loss: 1.2887 - accuracy: 0.4617 - val_loss: 1.7882 - val_accuracy: 0.4861 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - ETA: 0s - loss: 1.1402 - accuracy: 0.4829
Epoch 4: val_accuracy did not improve from 0.48611
20/20 [==============================] - 11s 535ms/s

...

20/20 [==============================] - ETA: 0s - loss: 2.4629 - accuracy: 0.3491
Epoch 1: val_accuracy improved from -inf to 0.47222, saving model to EfficientNetB2_model.h5
20/20 [==============================] - 76s 971ms/step - loss: 2.4629 - accuracy: 0.3491 - val_loss: 1.3861 - val_accuracy: 0.4722 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 1.1314 - accuracy: 0.4992
Epoch 2: val_accuracy improved from 0.47222 to 0.50000, saving model to EfficientNetB2_model.h5
20/20 [==============================] - 15s 740ms/step - loss: 1.1314 - accuracy: 0.4992 - val_loss: 1.0165 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 0.9583 - accuracy: 0.5661
Epoch 3: val_accuracy did not improve from 0.50000
20/20 [==============================] - 14s 649ms/step - loss: 0.9583 - accuracy: 0.5661 - val_loss: 1.0151 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/100
20/20 [==============================] -

2023-12-25 14:55:06.723560: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_10/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


20/20 [==============================] - ETA: 0s - loss: 5.5981 - accuracy: 0.4192
Epoch 1: val_accuracy improved from -inf to 0.44444, saving model to EfficientNetB3_model.h5
20/20 [==============================] - 90s 1s/step - loss: 5.5981 - accuracy: 0.4192 - val_loss: 2.2370 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 3.5893 - accuracy: 0.4568
Epoch 2: val_accuracy improved from 0.44444 to 0.45833, saving model to EfficientNetB3_model.h5
20/20 [==============================] - 22s 1s/step - loss: 3.5893 - accuracy: 0.4568 - val_loss: 1.8713 - val_accuracy: 0.4583 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 2.0654 - accuracy: 0.4959
Epoch 3: val_accuracy improved from 0.45833 to 0.55556, saving model to EfficientNetB3_model.h5
20/20 [==============================] - 22s 1s/step - loss: 2.0654 - accuracy: 0.4959 - val_loss: 0.9034 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 4/100
20/2

2023-12-25 15:10:32.927151: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_11/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


ResourceExhaustedError: Graph execution error:

Detected at node 'model_11/block3c_bn/FusedBatchNormV3' defined at (most recent call last):
    File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    Fi

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import tensorflow.keras
import os

# Function to evaluate the model and generate necessary metrics
def evaluate_model(model, test_generator, model_name):
    # Evaluate the model
    loss, accuracy = model.evaluate(test_generator, steps=np.ceil(test_generator.samples/test_generator.batch_size))

    # Generate predictions
    test_generator.reset()
    predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples/test_generator.batch_size))
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes

    # Compute confusion matrix and classification report
    cm = confusion_matrix(true_classes, predicted_classes)
    report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys(), output_dict=True)

    # Extract overall precision, recall, and f1-score
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    return accuracy, loss, precision, recall, f1_score, cm

# Path to save the CSV file
csv_file = 'model_results.csv'

# Check if the CSV file exists. If not, create it with appropriate headers
if not os.path.exists(csv_file):
    pd.DataFrame(columns=['Model', 'Accuracy', 'Loss', 'Precision', 'Recall', 'F1 Score']).to_csv(csv_file, index=False)

# Evaluate each model and append results to the CSV file
for model_name in model_info.keys():
    
    model_class, preprocess_input, input_shape = model_info[model_name]
    
    test_datagen = ImageDataGenerator(
        dtype='float32',
        preprocessing_function=preprocess_input,
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_folder,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False,
    )
    
    model = tensorflow.keras.models.load_model(f'{model_name}_model.h5')
    accuracy, loss, precision, recall, f1_score, cm = evaluate_model(model, test_generator, model_name)

    # Append results to the CSV
    new_row = pd.DataFrame([[model_name, accuracy, loss, precision, recall, f1_score]], columns=['Model', 'Accuracy', 'Loss', 'Precision', 'Recall', 'F1 Score'])
    pd.concat([pd.read_csv(csv_file), new_row]).to_csv(csv_file, index=False)

    # Print Confusion Matrix
    print(f'Confusion Matrix for {model_name}:')
    print(cm)
